___

<a href='http://www.pieriandata.com'> <img src='../Pierian_Data_Logo.png' /></a>

<center><em>Copyright: Pierian Data Inc.</em></center>

_____

In thise notebook we will take a look at **UniverseSelection** <br />
The idea behind universe selection is to compare multiple stocks based on some fundamentals, like the dividend yield.

In Quantconnect you can add a universe to your algorithm by using **AddUniverse(coarse_filter)**. <br />
The coarse_filter function gets a list of coarse fundamental objects as its argument (e.g price or volume. The full list can be found here: https://www.quantconnect.com/docs/algorithm-reference/universes#Universes-Coarse-Universe-Selection by clicking on "CoarseFundamental" written in red).
This filter allows you to filter out all stocks which meet a certain criteria.<br />
The global universe Resolution is defined via **self.UniverseSettings.Resolution**

Our first task will be to select the three most expensive stocks of all available data.
(If you do not add any Equities, quantconnect will automatically cover all available assets)

In [ ]:
class StockPrices(QCAlgorithm):
    def Initialize(self):
        
        self.SetStartDate(2019, 7, 12)
        self.SetCash(10000)
        
        # Globally sets the universe Resolution
        self.UniverseSettings.Resolution = Resolution.Daily
        
        self.AddUniverse(self.CoarseSelection)
    
    def CoarseSelection(self, coarse):
        # Sort all stocks by price descendingly
        sorted_by_price = sorted(coarse, key=lambda x: x.Price, reverse=True)

        # Return the symbol of the 3 most expensive stocks
        self.sorted_by_price = [x.Symbol for x in sorted_by_price][:4]
        
        # This automatically adds those stocks to self.Securities, so you access them in OnData
        return self.sorted_by_price

    def OnData(self, data):
        
        self.Log(self.Time)
        
        # Loop through the securites (3 most expensive stocks) and print their tickers + current price
        for sec in self.Securities.Values:
            # Check if data is available
            if not data.ContainsKey(sec.Symbol) or not data[sec.Symbol]:
                return

            self.Log(f"{data[sec.Symbol].Symbol}: {data[sec.Symbol].Open}$")
            
        self.Log("-------------------------------")
    

The **OnSecuritiesChanged(changes)** functions automatically checks for all changes in the Universe.
You can access the removed securities via **RemovedSecurities** and the added securities via **AddedSecurities**

In [ ]:
class StockPrices(QCAlgorithm):
    def Initialize(self):
        
        self.SetStartDate(2019, 7, 12)
        self.SetCash(10000)
        
        # Globally sets the universe Resolution
        self.UniverseSettings.Resolution = Resolution.Daily
        
        self.AddUniverse(self.CoarseSelection)
    
    def CoarseSelection(self, coarse):
        # Sort all stocks by price descendingly
        sorted_by_price = sorted(coarse, key=lambda x: x.Price, reverse=True)

        # Return the symbol of the 3 most expensive stocks
        self.sorted_by_price = [x.Symbol for x in sorted_by_price][:4]
        
        # This automatically adds those stocks to self.Securities, so you access them in OnData
        return self.sorted_by_price

    def OnData(self, data):
        
        self.Log(self.Time)
        
        # Loop through the securites (3 most expensive stocks) and print their tickers + current price
        for sec in self.Securities.Values:
            # Check if data is available
            if not data.ContainsKey(sec.Symbol) or not data[sec.Symbol]:
                return

            self.Log(f"{data[sec.Symbol].Symbol}: {data[sec.Symbol].Open}$")
            
        self.Log("-------------------------------")    
    
    def OnSecuritiesChanged(self, changes):
        self.Log(changes)
        
        # Print removed securities
        for security in changes.RemovedSecurities:
            self.Log(f"Removed: {security}")
        
        # Print added securities
        for security in changes.AddedSecurities:
            self.Log(f"Added: {security}")
 

We can use Added- and RemovedSecurities for buying and selling those stocks

In [ ]:
class PriceTrading(QCAlgorithm):
    def Initialize(self):
        
        self.SetStartDate(2019, 7, 12)
        self.SetCash(10000)
        
        # Globally sets the universe Resolution
        self.UniverseSettings.Resolution = Resolution.Daily
        
        self.AddUniverse(self.CoarseSelection)
    
    def CoarseSelection(self, coarse):
        # Sort all stocks by price descendingly
        sorted_by_price = sorted(coarse, key=lambda x: x.Price, reverse=True)

        # Return the symbol of the 3 most expensive stocks
        self.sorted_by_price = [x.Symbol for x in sorted_by_price][:4]
        
        # This automatically adds those stocks to self.Securities, so you access them in OnData
        return self.sorted_by_price

    def OnData(self, data):
        
        self.Log(self.Time)
        
        # Loop through the securites (3 most expensive stocks) and print their tickers + current price
        for sec in self.Securities.Values:
            # Check if data is available
            if not data.ContainsKey(sec.Symbol) or not data[sec.Symbol]:
                return

            self.Log(f"{data[sec.Symbol].Symbol}: {data[sec.Symbol].Open}$")
            
        self.Log("-------------------------------")    
     
    
    def OnSecuritiesChanged(self, changes):
        self.Log(changes)
        
        # Sell removed securities
        for security in changes.RemovedSecurities:
            self.Liquidate(security.Symbol)
            self.Log(f"Sell: {security}")
        
        # Buy added securities
        for security in changes.AddedSecurities:
            self.SetHoldings(security.Symbol, 0.3)
            self.Log(f"Buy: {security}")
 

You can additionally pass a **FineSelection(self, fine)** to the Universe to further filter your data according to stock fundamentals. As an example, we can select the 3 stocks with the largest dividend yield over the last 12 months(**ValuationRatios.TrailingDividendYield**).<br /> 
A full list of all available fundamental data can be found here: https://www.quantconnect.com/docs/data-library/fundamentals <br />
To get meaningful results, we only inspect companies with a volume of over 1 Bio \\$ and a stock price of at least 20$.


In [ ]:
class DividendYield(QCAlgorithm):
    def Initialize(self):
        
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2020, 1, 10)

        self.SetCash(10000)
        
        self.UniverseSettings.Resolution = Resolution.Daily
        
        self.AddUniverse(self.CoarseSelection, self.FineSelection)
    
    def CoarseSelection(self, coarse):
        # Select all stocks according to above description
        self.filteredByPrice = [x.Symbol for x in coarse if x.HasFundamentalData and \
                                x.DollarVolume > 1000000000 and \
                                x.Price > 20]
        

        # This automatically adds those stocks to self.Securities, so you access them in OnData
        return self.filteredByPrice
        
    def FineSelection(self, fine):
        
        # Sort descendingly by Trailing Dividend Yield
        sorted_by_dividend = sorted(fine, key=lambda x: x.ValuationRatios.TrailingDividendYield, reverse=True)
        # Get corresponding symbols and actual dividend
        self.sorted_by_dividend = [x.Symbol for x in sorted_by_dividend]
        self.dividends = [x.Value for x in sorted_by_dividend]
        
        # Loop through the list to print the Symbol and dividend
        for i in range(4):
            self.Log(f"{self.sorted_by_dividend[i]}:  Yield: {self.dividends[i]} %")
        self.Log("-------------------------------")
        return self.sorted_by_dividend[:4]
        
        


We can also use this information for buying and selling:
Let's buy based on the EBIT (Earnings - expenses): **fine.FinancialStatements.IncomeStatement.EBIT.Period**. <br />
As you can see, this statement has an additional parameter called **perdiod** which defines how many months you want to go back and thus how many EBIT statements you want to take into consideration.<br />
Let's use the last twelve months for this example

In [ ]:
class EBITAlgorithm(QCAlgorithm):
    def Initialize(self):
        
        self.SetStartDate(2018, 1, 1)

        self.SetCash(10000)
        
        self.UniverseSettings.Resolution = Resolution.Daily
        
        self.AddUniverse(self.CoarseSelection, self.FineSelection)
    
    def CoarseSelection(self, coarse):
        # Select all stocks which have fundamental data
        self.filtered = [x.Symbol for x in coarse if x.HasFundamentalData]
        

        # This automatically adds those stocks to self.Securities, so you access them in OnData
        return self.filtered
        
    def FineSelection(self, fine):
        
        # Sort descendingly by EBIT and get the 3 largest
        sorted_by_ebit = sorted(fine, key=lambda x: x.FinancialStatements.IncomeStatement.EBIT.TwelveMonths, reverse=True)[:4]
        
        # Get corresponding symbols
        self.sorted_by_ebit = [x.Symbol for x in sorted_by_ebit]

        return self.sorted_by_ebit
        
        
    def OnData(self, data):
        
        self.Log(self.Time)
        
        # Loop thorugh the securites (three companies with largest EBIT) and print their tickers + current price
        for sec in self.Securities.Values:
            if not data.ContainsKey(sec.Symbol) or not data[sec.Symbol]:
                return
            self.Log(f"{data[sec.Symbol].Symbol}: {data[sec.Symbol].Open}$")
            
        self.Log("-------------------------------")
    
    
    def OnSecuritiesChanged(self, changes):
        self.Log(changes)
        
        # Sell removed securities
        for security in changes.RemovedSecurities:
            self.Liquidate(security.Symbol)
            self.Log(f"Sell: {security}")
        
        # Buy added securities
        for security in changes.AddedSecurities:
            self.SetHoldings(security.Symbol, 0.3)
            self.Log(f"Buy: {security}")


Another fundamental we can use for traing is the P/E ratio (Price/Earnings ratio) (https://en.wikipedia.org/wiki/Price%E2%80%93earnings_ratio).

Let's say that we invest all companies with a P/E ratio > 3000.<br />
Note that this backtest might need some minutes

In [ ]:
class PETrading(QCAlgorithm):
    def Initialize(self):
        
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2020, 6, 1)

        self.SetCash(10000)
        
        self.UniverseSettings.Resolution = Resolution.Daily
        
        self.AddUniverse(self.CoarseSelection, self.FineSelection)

    def CoarseSelection(self, coarse):

        # Select all stocks which have fundamental data and a 
        self.filtered = [x.Symbol for x in coarse if x.HasFundamentalData]

        return self.filtered
        
    def FineSelection(self, fine):

        # Get all companies with a P/E ratio > 3000
        self.filtered_by_pe = [x.Symbol for x in fine if x.ValuationRatios.PERatio > 3000]

        return self.filtered_by_pe


    def OnSecuritiesChanged(self, changes):
        self.Log(changes)
        
        # Sell removed securities
        for security in changes.RemovedSecurities:
            self.Liquidate(security.Symbol)
            self.Log(f"Sell: {security}")
        
        # Get the number of added changes
        for security in changes.AddedSecurities:
            self.SetHoldings(security.Symbol, 0.1)
            self.Log(f"Buy: {security}")
            


We can also combine multiple FineSelection fundamentals by concattenating them via **and** in the filter step

In [ ]:
class PETradingCombination(QCAlgorithm):
    def Initialize(self):
        
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2020, 3, 1)
        self.SetCash(10000)
        
        self.UniverseSettings.Resolution = Resolution.Daily
        
        self.AddUniverse(self.CoarseSelection, self.FineSelection)

    def CoarseSelection(self, coarse):

        
        # Select all stocks which have fundamental data and a 
        self.filtered = [x.Symbol for x in coarse if x.HasFundamentalData]

        return self.filtered
        
    def FineSelection(self, fine):

        # Get all companies with a P/E ratio > 1000 and a Trailing Dividend Ratio > 0.1
        self.filtered_by_pe = [x.Symbol for x in fine if x.ValuationRatios.PERatio > 1000 and \
                                                         x.ValuationRatios.TrailingDividendYield > 0.1]

        return self.filtered_by_pe


    def OnSecuritiesChanged(self, changes):
        self.Log(changes)
        
        # Sell removed securities
        for security in changes.RemovedSecurities:
            self.Liquidate(security.Symbol)
            self.Log(f"Sell: {security}")
        
        # Get the number of added changes
        for security in changes.AddedSecurities:
            self.SetHoldings(security.Symbol, 0.1)
            self.Log(f"Buy: {security}")
            
